In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_593429/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_37_7,0.999850,0.822114,0.795902,0.999475,0.000675,1.189527,0.730048,0.001086,0.075664,0.025987,1.001242,0.026439,144.600567,279.640501,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_37_8,0.999850,0.822100,0.795738,0.999451,0.000676,1.189619,0.730634,0.001136,0.075334,0.026007,1.001244,0.026460,144.597500,279.637434,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_37_6,0.999850,0.822122,0.796046,0.999492,0.000677,1.189472,0.729532,0.001050,0.076017,0.026027,1.001246,0.026479,144.594524,279.634458,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_37_9,0.999849,0.822081,0.795560,0.999425,0.000678,1.189744,0.731270,0.001188,0.075150,0.026038,1.001247,0.026491,144.592742,279.632676,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_37_5,0.999849,0.822137,0.796196,0.999511,0.000678,1.189368,0.728996,0.001010,0.076372,0.026043,1.001248,0.026496,144.591977,279.631911,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,model_1_1_4,0.948740,0.790439,0.687268,0.966949,0.230905,1.401333,1.118625,0.203242,1.159597,0.480526,1.424725,0.488883,132.931497,267.971431,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
486,model_1_1_3,0.939879,0.786816,0.674959,0.959046,0.270820,1.425566,1.162651,0.251838,1.245730,0.520404,1.498144,0.529455,132.612603,267.652537,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
487,model_1_1_2,0.938689,0.786223,0.673751,0.957701,0.276180,1.429525,1.166974,0.260110,1.255617,0.525528,1.508003,0.534668,132.573406,267.613340,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
488,model_1_1_1,0.936174,0.784959,0.670839,0.955294,0.287512,1.437979,1.177391,0.274911,1.278581,0.536202,1.528847,0.545528,132.492980,267.532913,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
